In [74]:
import argparse
import datetime
import pandas as pd
#from utils import perform_get_request, xml_to_load_dataframe, xml_to_gen_data

In [75]:
import requests
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime, timedelta

def xml_to_gen_data(xml_data) -> dict:
    """
    Parse the XML data of generation into a dictionary of DataFrames, one for each PsrType.
    """

    # Define the XML namespace
    namespace = {'ns': 'urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0'}
    
    # Parse the XML data
    root = ET.fromstring(xml_data)
    
    # Get all TimeSeries tags
    time_series_tags = root.findall('.//ns:TimeSeries', namespace)
    
    # Initialize a dictionary to hold the data
    data = {"StartTime": [], "EndTime": [], "AreaID": [], "UnitName": [], "PsrType": [], "quantity": []}

    # Loop over all TimeSeries tags
    for ts in time_series_tags:
        # Extract PsrType from MktPSRType if it exists
        psr_type_tag = ts.find('ns:MktPSRType/ns:psrType', namespace)
        psr_type = psr_type_tag.text if psr_type_tag is not None else None

        # Extract AreaID and UnitName if they exist
        area_id_tag = ts.find('ns:inBiddingZone_Domain.mRID', namespace)
        area_id = area_id_tag.text if area_id_tag is not None else None
        unit_name_tag = ts.find('ns:quantity_Measure_Unit.name', namespace)
        unit_name = unit_name_tag.text if unit_name_tag is not None else None

        # Extract the time period start and end if it exists
        time_period = ts.find('ns:Period', namespace)
        if time_period is not None:
            period_start = time_period.find('ns:timeInterval/ns:start', namespace).text
            period_end = time_period.find('ns:timeInterval/ns:end', namespace).text
            resolution = time_period.find('ns:resolution', namespace).text

            # Resolution is PT15M or PT60M
            resolution_minutes = int(resolution.replace('PT', '').replace('M', ''))

            # Extract the point values
            points = time_period.findall('ns:Point', namespace)
            for point in points:
                position = point.find('ns:position', namespace).text
                quantity = point.find('ns:quantity', namespace).text

                # Calculate the actual start and end time for each resolution_minutes interval
                start_time_interval = datetime.fromisoformat(period_start.replace('Z', '+00:00'))
                end_time_interval = start_time_interval + timedelta(minutes=resolution_minutes*(int(position)-1))
                start_time_interval = end_time_interval - timedelta(minutes=resolution_minutes)

                # Append the StartTime, EndTime, AreaID, UnitName, PsrType, and quantity values to the data dictionary
                data["StartTime"].append(start_time_interval.isoformat(timespec='minutes')+'Z')
                data["EndTime"].append(end_time_interval.isoformat(timespec='minutes')+'Z')
                data["AreaID"].append(area_id)
                data["UnitName"].append(unit_name)
                data["PsrType"].append(psr_type)
                data["quantity"].append(quantity)

    # Convert the data dictionary into a pandas DataFrame
    df = pd.DataFrame(data)

    # Create a separate DataFrame for each PsrType
    df_dict = {psr_type: df[df["PsrType"] == psr_type] for psr_type in df["PsrType"].unique()}
    
    return df_dict

def xml_to_load_dataframe(xml_data) -> pd.DataFrame:
    """
    Parse the XML data of Load into a pandas DataFrame.
    """
    namespace = {'ns': 'urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0'}
    root = ET.fromstring(xml_data)

    data = []
    for time_series in root.findall('.//ns:TimeSeries', namespace):
        series_id = time_series.find('ns:mRID', namespace).text
        business_type = time_series.find('ns:businessType', namespace).text
        object_aggregation = time_series.find('ns:objectAggregation', namespace).text
        domain_mrid = time_series.find('ns:outBiddingZone_Domain.mRID', namespace).text
        unit_name = time_series.find('ns:quantity_Measure_Unit.name', namespace).text
        curve_type = time_series.find('ns:curveType', namespace).text

        for period in time_series.findall('ns:Period', namespace):
            start_time = period.find('ns:timeInterval/ns:start', namespace).text
            end_time = period.find('ns:timeInterval/ns:end', namespace).text
            resolution = period.find('ns:resolution', namespace).text

            # Resolution is PT15M or PT60M
            resolution_minutes = int(resolution.replace('PT', '').replace('M', ''))
            
            for point in period.findall('ns:Point', namespace):
                position = point.find('ns:position', namespace).text
                quantity = point.find('ns:quantity', namespace).text

                # calculate the actual start and end time for each resolution_minutes interval
                start_time_interval = datetime.fromisoformat(start_time.replace('Z', '+00:00'))
                end_time_interval = start_time_interval + timedelta(minutes=resolution_minutes*(int(position)-1))
                start_time_interval = end_time_interval - timedelta(minutes=resolution_minutes)

                data.append([start_time_interval.isoformat(timespec='minutes')+'Z', end_time_interval.isoformat(timespec='minutes')+'Z', 
                             domain_mrid, unit_name, quantity])

    df = pd.DataFrame(data, columns=['StartTime', 'EndTime', 'AreaID', 'UnitName', 'Load'])
    return df

def make_url(base_url, params):
    query_string = "&".join([f"{k}={v}" for k, v in params.items()])
    return f"{base_url}?{query_string}"

def perform_get_request(base_url, params):
    url = make_url(base_url, params)
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return response.content

In [76]:
def get_load_data_from_entsoe(regions, periodStart='202302240000', periodEnd='202303240000', output_path='/kaggle/working/'):
    
    # TODO: There is a period range limit of 1 year for this API. Process in 1 year chunks if needed
    
    # URL of the RESTful API
    url = 'https://web-api.tp.entsoe.eu/api'

    # General parameters for the API
    # Refer to https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html#_documenttype
    params = {
        'securityToken': '1d9cd4bd-f8aa-476c-8cc1-3442dc91506d',
        'documentType': 'A65',
        'processType': 'A16',
        'outBiddingZone_Domain': 'FILL_IN', # used for Load data
        'periodStart': periodStart, # in the format YYYYMMDDHHMM
        'periodEnd': periodEnd # in the format YYYYMMDDHHMM
    }

    # Loop through the regions and get data for each region
    for region, area_code in regions.items():
        print(f'Fetching data for {region}...')
        params['outBiddingZone_Domain'] = area_code
    
        # Use the requests library to get data from the API for the specified time range
        response_content = perform_get_request(url, params)

        # Response content is a string of XML data
        #df = xml_to_load_dataframe(response_content, 'Load')
        df = xml_to_load_dataframe(response_content)

        # Save the DataFrame to a CSV file
        df.to_csv(f'{output_path}/load_{region}.csv', index=False)
       
    return

In [77]:
def get_gen_data_from_entsoe(regions, periodStart='202302240000', periodEnd='202303240000', output_path='/kaggle/working/'):
    
    # TODO: There is a period range limit of 1 day for this API. Process in 1 day chunks if needed

    # URL of the RESTful API
    url = 'https://web-api.tp.entsoe.eu/api'

    # General parameters for the API
    params = {
        'securityToken': '1d9cd4bd-f8aa-476c-8cc1-3442dc91506d',
        'documentType': 'A75',
        'processType': 'A16',
        'outBiddingZone_Domain': 'FILL_IN', # used for Load data
        'in_Domain': 'FILL_IN', # used for Generation data
        'periodStart': periodStart, # in the format YYYYMMDDHHMM
        'periodEnd': periodEnd # in the format YYYYMMDDHHMM
    }

    # Loop through the regions and get data for each region
    for region, area_code in regions.items():
        print(f'Fetching data for {region}...')
        params['outBiddingZone_Domain'] = area_code
        params['in_Domain'] = area_code
    
        # Use the requests library to get data from the API for the specified time range
        response_content = perform_get_request(url, params)

        # Response content is a string of XML data
        dfs = xml_to_gen_data(response_content)

        # Save the dfs to CSV files
        for psr_type, df in dfs.items():
            # Save the DataFrame to a CSV file
            df.to_csv(f'{output_path}/gen_{region}_{psr_type}.csv', index=False)
    
    return

In [78]:
def parse_arguments():
    parser = argparse.ArgumentParser(description='Data ingestion script for Energy Forecasting Hackathon')
    parser.add_argument(
        '--start_time', 
        type=lambda s: datetime.datetime.strptime(s, '%Y-%m-%d'), 
        default=datetime.datetime(2023, 1, 1), 
        help='Start time for the data to download, format: YYYY-MM-DD'
    )
    parser.add_argument(
        '--end_time', 
        type=lambda s: datetime.datetime.strptime(s, '%Y-%m-%d'), 
        default=datetime.datetime(2023, 1, 2), 
        help='End time for the data to download, format: YYYY-MM-DD'
    )
    parser.add_argument(
        '--output_path', 
        type=str, 
        default='./data',
        help='Name of the output file'
    )
    return parser.parse_args()

In [79]:
def main(start_time, end_time, output_path):
    
    regions = {
        'HU': '10YHU-MAVIR----U',
        'IT': '10YIT-GRTN-----B',
        'PO': '10YPL-AREA-----S',
        'SP': '10YES-REE------0',
        'UK': '10Y1001A1001A92E',
        'DE': '10Y1001A1001A83F',
        'DK': '10Y1001A1001A65H',
        'SE': '10YSE-1--------K',
        'NE': '10YNL----------L',
    }

    # Transform start_time and end_time to the format required by the API: YYYYMMDDHHMM
    start_time = start_time.strftime('%Y%m%d%H%M')
    end_time = end_time.strftime('%Y%m%d%H%M')

    # Get Load data from ENTSO-E
    get_load_data_from_entsoe(regions, start_time, end_time, output_path)

    # Get Generation data from ENTSO-E
    get_gen_data_from_entsoe(regions, start_time, end_time, output_path)

In [ ]:
if __name__ == "__main__":
    args = parse_arguments()
    main(args.start_time, args.end_time, args.output_path)

In [80]:
regions = {
    #'HU': '10YHU-MAVIR----U',
    #'IT': '10YIT-GRTN-----B',
    #'PO': '10YPL-AREA-----S',
    #'SP': '10YES-REE------0',
    #'UK': '10Y1001A1001A92E',
    'DE': '10Y1001A1001A83F',
    #'DK': '10Y1001A1001A65H',
    #'SE': '10YSE-1--------K',
    #'NE': '10YNL----------L',
}

# Get Load data from ENTSO-E
#get_load_data_from_entsoe(regions, start_time, end_time, output_path)
get_load_data_from_entsoe(regions, '202201010000', '202301010000')

# Get Generation data from ENTSO-E
#get_gen_data_from_entsoe(regions, start_time, end_time, output_path)
get_gen_data_from_entsoe(regions, '202201010000', '202301010000')

Fetching data for DE...
Fetching data for DE...


In [10]:
import pandas as pd
import numpy as np

In [92]:
file_name = "uk/gen_UK_B19.csv"

In [93]:
df_gen = pd.read_csv(f"/kaggle/input/europe-green-electricity-generation-consumption/2022-2023/{file_name}")
df_gen

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
0,2022-01-26T23:30+00:00Z,2022-01-27T00:00+00:00Z,10Y1001A1001A92E,MAW,B19,433
1,2022-01-27T00:00+00:00Z,2022-01-27T00:30+00:00Z,10Y1001A1001A92E,MAW,B19,472
2,2022-01-27T00:30+00:00Z,2022-01-27T01:00+00:00Z,10Y1001A1001A92E,MAW,B19,549
3,2022-01-27T01:00+00:00Z,2022-01-27T01:30+00:00Z,10Y1001A1001A92E,MAW,B19,520
4,2022-01-27T01:30+00:00Z,2022-01-27T02:00+00:00Z,10Y1001A1001A92E,MAW,B19,407
...,...,...,...,...,...,...
12639,2022-12-31T21:00+00:00Z,2022-12-31T21:30+00:00Z,NaN,MAW,B19,183
12640,2022-12-31T21:30+00:00Z,2022-12-31T22:00+00:00Z,NaN,MAW,B19,155
12641,2022-12-31T22:00+00:00Z,2022-12-31T22:30+00:00Z,NaN,MAW,B19,132
12642,2022-12-31T22:30+00:00Z,2022-12-31T23:00+00:00Z,NaN,MAW,B19,112


In [94]:
df_gen.dtypes

StartTime    object
EndTime      object
AreaID       object
UnitName     object
PsrType      object
quantity      int64
dtype: object

In [95]:
df_gen = df_gen.drop(labels=["AreaID", "UnitName", "PsrType"], axis=1)
df_gen

,StartTime,EndTime,quantity
0,2022-01-26T23:30+00:00Z,2022-01-27T00:00+00:00Z,433
1,2022-01-27T00:00+00:00Z,2022-01-27T00:30+00:00Z,472
2,2022-01-27T00:30+00:00Z,2022-01-27T01:00+00:00Z,549
3,2022-01-27T01:00+00:00Z,2022-01-27T01:30+00:00Z,520
4,2022-01-27T01:30+00:00Z,2022-01-27T02:00+00:00Z,407
...,...,...,...
12639,2022-12-31T21:00+00:00Z,2022-12-31T21:30+00:00Z,183
12640,2022-12-31T21:30+00:00Z,2022-12-31T22:00+00:00Z,155
12641,2022-12-31T22:00+00:00Z,2022-12-31T22:30+00:00Z,132
12642,2022-12-31T22:30+00:00Z,2022-12-31T23:00+00:00Z,112


In [96]:
from datetime import datetime, timedelta, timezone

df_gen_resampling = pd.DataFrame(data=[], columns=["StartTime", "EndTime", "quantity"])

#num_time_slot_in_hour = 4
num_time_slot_in_hour = 2
#interval_min_in_hour = 15
interval_min_in_hour = 30
interval_count = 0
quantity = 0
index = 0
indx = 0
start_date_1 = datetime.now()
start_date = datetime.now()
pre_end_date = datetime.now()
next_start_date = datetime.now()
interval_min = timedelta(minutes=interval_min_in_hour)
interval_hour = timedelta(hours=1)

while index < df_gen.shape[0]:
    row = df_gen.loc[index]
    index = index + 1
    if interval_count == num_time_slot_in_hour: 
        start_date_1 = datetime(start_date_1.year, start_date_1.month, start_date_1.day, start_date_1.hour, tzinfo=start_date_1.tzinfo)
        df_gen_resampling.loc[indx, "StartTime"] = start_date_1
        df_gen_resampling.loc[indx, "EndTime"] = start_date_1 + interval_hour
        df_gen_resampling.loc[indx, "quantity"] = quantity
        interval_count = 0
        quantity = 0
        indx = indx + 1
    
    start_date = datetime.fromisoformat(str(row["StartTime"]).replace("Z", ""))
    row_end_date = datetime.fromisoformat(str(row["EndTime"]).replace("Z", ""))
    end_date = start_date + interval_min
    
    if interval_count == 0:
        start_date_1 = start_date
        interval_count = interval_count + 1
        if end_date == row_end_date:
            quantity = quantity + row["quantity"]
    else:
        iter_count = 0
        while interval_count < num_time_slot_in_hour:
            if iter_count == 0:
                pre_end_date = datetime.fromisoformat(str(df_gen.loc[index - 2, "EndTime"]).replace("Z", ""))
            else:
                pre_end_date = pre_end_date + interval_min
            iter_count = iter_count + 1
            pre_start_date = pre_end_date - interval_min
            interval_count = interval_count + 1
            if start_date == pre_end_date and start_date.hour == pre_start_date.hour:
                if end_date > pre_end_date and end_date == row_end_date:
                    quantity = quantity + row["quantity"]
                    break
            if interval_count == num_time_slot_in_hour:
                index = index - 1

In [97]:
start_date_1 = datetime(start_date_1.year, start_date_1.month, start_date_1.day, start_date_1.hour, tzinfo=start_date_1.tzinfo)
df_gen_resampling.loc[indx, "StartTime"] = start_date_1
df_gen_resampling.loc[indx, "EndTime"] = start_date_1 + interval_hour
df_gen_resampling.loc[indx, "quantity"] = quantity

In [98]:
df_gen_resampling

,StartTime,EndTime,quantity
0,2022-01-26 23:00:00+00:00,2022-01-27 00:00:00+00:00,433
1,2022-01-27 00:00:00+00:00,2022-01-27 01:00:00+00:00,1021
2,2022-01-27 01:00:00+00:00,2022-01-27 02:00:00+00:00,927
3,2022-01-27 02:00:00+00:00,2022-01-27 03:00:00+00:00,673
4,2022-01-27 03:00:00+00:00,2022-01-27 04:00:00+00:00,426
...,...,...,...
6463,2022-12-31 19:00:00+00:00,2022-12-31 20:00:00+00:00,412
6464,2022-12-31 20:00:00+00:00,2022-12-31 21:00:00+00:00,415
6465,2022-12-31 21:00:00+00:00,2022-12-31 22:00:00+00:00,338
6466,2022-12-31 22:00:00+00:00,2022-12-31 23:00:00+00:00,244


In [99]:
file_name = "gen_UK_B19_resampling.csv"
df_gen_resampling.to_csv(f"/kaggle/working/{file_name}", index=False)